# Лабораторная работа 5. Классификация изображений с помощью сверточной нейронной сети

### Работу выполнил:<span style="color:blue"> {ваше имя и фамилия}</span>

### Сделанную лабораторную работу отправляйте через [ФОРМУ](https://vyatsu-my.sharepoint.com/:f:/g/personal/usr09019_vyatsu_ru/Ei2kQNkh2ABHuYFR4BPtyrEBRGDgM28MZCQVhyrDQOEOGA)

## Задание 1.

Построить сверточную нейронную сеть для классификации изображений на основе датасета Intel Image Classification (https://www.kaggle.com/datasets/puneet6060/intel-image-classification). Задание предполагает самостоятельное создание структуры сети и обучение её с нуля.

В работе можно ориентироваться на структуру сети [AlexNet](https://en.wikipedia.org/wiki/AlexNet).

Ниже приводится пример создания сети с использованием библиотеки PyTorch. Данный пример не является готовым решением: сеть переобучилась и на тестовых данных показывает низкое значение метрик качества (accuracy/loss).

Задание считается выполненным при достижении accuracy=0.83 на тестовом наборе данных.

In [ ]:
# Импортируем необходимые модули
import torch
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt

In [ ]:
# Директории обучающих и тестовых данных
data_dir = "../data/seg_train/seg_train"
test_data_dir = "../data/seg_test/seg_test"

In [ ]:
# Улучшенные преобразования с аугментацией для обучающих данных
train_transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.RandomHorizontalFlip(p=0.5),  # Случайное отражение по горизонтали
    transforms.RandomRotation(degrees=15),   # Случайный поворот на ±15 градусов
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # Изменение цветовых характеристик
    transforms.ToTensor(),
])

# Преобразования для валидационных и тестовых данных (без аугментации)
val_test_transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor()
])

# Загружаем данные с разными преобразованиями
dataset = ImageFolder(data_dir, transform=train_transform)
test_dataset = ImageFolder(test_data_dir, transform=val_test_transform)

In [ ]:
# Посмотрим на размерность данных и метки
img, label = dataset[0]
print(img.shape, label)

In [ ]:
print("Классы изображений : \n", dataset.classes)

In [ ]:
# Определим функцию для отображения изображений датасета
def display_img(img, label):
    print(f"Label : {dataset.classes[label]}")
    plt.imshow(img.permute(1,2,0))  # Преобразуем (3, 150, 150) в (150, 150, 3)

# Отобразим первое изображение датасета
display_img(*dataset[0])

In [ ]:
# Определяем структуру нейронной сети с Batch Normalization и Dropout
import torch.nn as nn

class ConvNet(nn.Module):
    def __init__(self, numChannels, classes):
        # Вызываем родительский конструктор
        super(ConvNet, self).__init__()

        # Первый набор слоев CONV => BN => RELU => POOL
        self.conv1 = nn.Conv2d(in_channels=numChannels, out_channels=32, kernel_size=(3, 3), padding=(1, 1))
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

        # Второй набор слоев CONV => BN => RELU => POOL
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=(1, 1))
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

        # Третий набор слоев CONV => BN => RELU => POOL
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=(1, 1))
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.dropout3 = nn.Dropout(0.25)  # Dropout после третьего сверточного блока

        # Первый набор слоев FC => BN => RELU => DROPOUT
        self.fc1 = nn.Linear(in_features=41472, out_features=1024)
        self.bn4 = nn.BatchNorm1d(1024)
        self.relu4 = nn.ReLU()
        self.dropout4 = nn.Dropout(0.5)  # Более высокий dropout для полносвязных слоев

        # Второй набор слоев FC => BN => RELU => DROPOUT
        self.fc2 = nn.Linear(in_features=1024, out_features=512)
        self.bn5 = nn.BatchNorm1d(512)
        self.relu5 = nn.ReLU()

        # Инициализируем классификатор softmax
        self.fc3 = nn.Linear(in_features=512, out_features=classes)
        self.logSoftmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        # Первый блок: CONV => BN => RELU => POOL => DROPOUT
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)

        # Второй блок: CONV => BN => RELU => POOL => DROPOUT
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)

        # Третий блок: CONV => BN => RELU => POOL => DROPOUT
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.maxpool3(x)
        x = self.dropout3(x)

        # Переводим результат предыдущего слоя в одномерный вид
        x = torch.flatten(x, 1)

        # Полносвязные слои с BatchNorm и Dropout
        x = self.fc1(x)
        x = self.bn4(x)
        x = self.relu4(x)
        x = self.dropout4(x)

        x = self.fc2(x)
        x = self.bn5(x)
        x = self.relu5(x)

        # Классификатор (без dropout)
        x = self.fc3(x)
        output = self.logSoftmax(x)

        return output

In [ ]:
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split

# Определяем гиперпараметры
BATCH_SIZE = 32
INIT_LR = 1e-3
EPOCHS = 15
TRAIN_SPLIT = 0.75
VAL_SPLIT = 1 - TRAIN_SPLIT

train_size = int(len(dataset) * TRAIN_SPLIT)
val_size = len(dataset) - train_size

# Определяем устройство, которое будет использоваться для обучения модели
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_data, val_data = random_split(dataset, [train_size, val_size])

print(f"Размер обучающих данных: {len(train_data)}")
print(f"Размер валидационных данных : {len(val_data)}")

# Загружаем обучающие, валидационные и тестовые данные в батчи
train_dl = DataLoader(train_data, BATCH_SIZE, shuffle=True, num_workers=0)
val_dl = DataLoader(val_data, BATCH_SIZE, num_workers=0)
test_dl = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Вычисляем количество шагов на одну эпоху для обучающего и валидационного набора
train_steps = len(train_dl.dataset) // BATCH_SIZE
val_steps = len(val_dl.dataset) // BATCH_SIZE

In [ ]:
from torch.optim import Adam
import time

# Инициализируем модель нейронной сети
model = ConvNet(
	numChannels=3,
	classes=len(dataset.classes)).to(device)
# Инициализируем оптимизатор и функцию потерь
opt = Adam(model.parameters(), lr=INIT_LR, weight_decay=1e-4)
loss_fn = nn.NLLLoss()
# Инициализируем словарь для сохранения истории обучения
H = {
	"train_loss": [],
	"train_acc": [],
	"val_loss": [],
	"val_acc": []
}

In [ ]:
# Будем замерять, как долго длился процесс обучения
start_time = time.time()

# Цикл по эпохам
for e in range(0, EPOCHS):
	# Переводим модель в режим обучения
	model.train()
	# Переменные для хранения общих потерь обучения и валидации
	total_train_loss = 0
	total_val_loss = 0
	# Количество корректных предсказаний на шаге обучения
	# и валидации
	train_сorrect = 0
	val_сorrect = 0
	# Цикл по обучающему множеству
	for (x, y) in train_dl:
		# Посылаем данные на устройство
		(x, y) = (x.to(device), y.to(device))
		# Выполняем прямой проход и считаем потери при обучении
		pred = model(x)
		loss = loss_fn(pred, y)
		# Обнуляем градиенты, выполняем шаг обратного распространения ошибки,
		# и обновляем веса
		opt.zero_grad()
		loss.backward()
		opt.step()
		# Добавляем потери к общим потерям и
		# вычисляем количество правильных предсказаний
		total_train_loss += loss
		train_сorrect += (pred.argmax(1) == y).type(
			torch.float).sum().item()

	# Отключаем автоградиент для определения качества
	with torch.no_grad():
		# Переводим модель в режим определения качества
		model.eval()
		# Цикл по валидационному множеству
		for (x, y) in val_dl:
			# Посылаем данные на устройство
			(x, y) = (x.to(device), y.to(device))
			# Выполняем предсказания и находим потери при валидации
			pred = model(x)
			total_val_loss += loss_fn(pred, y)
			# Вычисляем количество правильных предсказаний
			val_сorrect += (pred.argmax(1) == y).type(
				torch.float).sum().item()

	# Вычисляем средние потери при обучении и валидации
	avg_train_loss = total_train_loss / train_steps
	avg_val_loss = total_val_loss / val_steps
	# Вычисляем accuracy при обучении и валидации
	train_сorrect = train_сorrect / len(train_dl.dataset)
	val_сorrect = val_сorrect / len(val_dl.dataset)
	# Обновляем историю обучения
	H["train_loss"].append(avg_train_loss.cpu().detach().numpy())
	H["train_acc"].append(train_сorrect)
	H["val_loss"].append(avg_val_loss.cpu().detach().numpy())
	H["val_acc"].append(val_сorrect)
	# Выводим информации оо обучении и валидации модели
	print("[INFO] EPOCH: {}/{}".format(e + 1, EPOCHS))
	print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(
		avg_train_loss, train_сorrect))
	print("Val loss: {:.6f}, Val accuracy: {:.4f}\n".format(
		avg_val_loss, val_сorrect))

end_time = time.time()
print("[INFO] total time taken to train the model: {:.2f}s".format(
	end_time - start_time))

In [ ]:
# Сейчас мы можем оценить качество модели на тестовом наборе данных
from sklearn.metrics import classification_report
import numpy as np

# Отключаем автоградиент для определения качества
with torch.no_grad():
	# Переводим модель в режим определения качества
	model.eval()

	# Список для хранения предсказаний
	preds = []
	# Цикл по тестовому набору данных
	for (x, y) in test_dl:
		# Посылаем данные на устройство
		x = x.to(device)
		# Делаем предсказание и помещаем в список
		pred = model(x)
		preds.extend(pred.argmax(axis=1).cpu().numpy())
# Генерируем отчёт о классификации
print(classification_report(test_dataset.targets, np.array(preds), target_names=test_dataset.classes))

In [ ]:
# График потерь и accuracy при обучении
plt.style.use("ggplot")
plt.figure()
plt.plot(H["train_loss"], label="train_loss")
plt.plot(H["val_loss"], label="val_loss")
plt.plot(H["train_acc"], label="train_acc")
plt.plot(H["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")

# Сохраняем модель на диск
torch.save(model, "cnn.pt")

## Задание 2.

Написать функцию, выполняющую предсказание класса для входного изображения. На вход функция принимает имя файла, а возвращает предсказанный класс.

In [50]:
import torch
from PIL import Image

def predict_image_class(model_path, image_path, class_names):

    model = torch.load(model_path, warn=True)
    model.eval()

    transform = transforms.Compose([
        transforms.Resize((150, 150)),
        transforms.ToTensor()
    ])

    # Загружаем и преобразуем изображение
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)  # Добавляем размерность батча

    # Предсказание
    with torch.no_grad():
        output = model(image)
        predicted_class = torch.argmax(output, 1).item()

    return class_names[predicted_class]

# Пример использования
class_names = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']
predicted_class = predict_image_class("cnn.pt", "example_image.jpg", class_names)
print(f"Предсказанный класс: {predicted_class}")

TypeError: Unpickler.__init__() got an unexpected keyword argument 'warn'

## Задание 3.

Выполнить задание 2, используя предобученную сеть AlexNet, доступную в PyTorch ([ссылка](https://pytorch.org/hub/pytorch_vision_alexnet/)). Выполнять дообучение сети не нужно.

In [ ]:
# Ваш код!